# Data Preprocessing & Feature Engineering

**Objective:** Prepare cleaned data for modeling by handling missing values, creating new features, encoding categorical variables, and splitting data.

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Load cleaned data from notebook 1
df = pd.read_csv('../data/processed/loans_cleaned_final.csv')

print(f"Data loaded: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

Data loaded: (87889, 91)
Rows: 87,889
Columns: 91


In [2]:
# Identify columns with missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percentage': missing_pct.values
})

missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print(f"Columns with missing values: {len(missing_df)}")
print("\nMissing values summary:")
print(missing_df)

Columns with missing values: 14

Missing values summary:
                    Column  Missing_Count  Missing_Percentage
24  mths_since_last_delinq          42231           48.050382
64   mths_since_recent_inq           9268           10.545119
8                emp_title           5648            6.426288
9               emp_length           5627            6.402394
75        num_tl_120dpd_2m           4590            5.222497
58      mo_sin_old_il_acct           2560            2.912765
80        percent_bc_gt_75            971            1.104803
55                 bc_util            944            1.074082
54          bc_open_to_buy            896            1.019468
63    mths_since_recent_bc            858            0.976231
40            last_pymnt_d             71            0.080784
28              revol_util             36            0.040961
42      last_credit_pull_d              3            0.003413
72           num_rev_accts              1            0.001138


In [3]:
# Strategy for each column:

# 1. mths_since_last_delinq (48% missing) - Missing means NO delinquency
df['mths_since_last_delinq'].fillna(999, inplace=True)
df['has_delinq'] = (df['mths_since_last_delinq'] != 999).astype(int)

# 2. mths_since_recent_inq (10.5% missing) - Missing means no recent inquiry
df['mths_since_recent_inq'].fillna(999, inplace=True)

# 3. emp_title (6.4% missing) - Drop this column (too many unique values, not useful)
if 'emp_title' in df.columns:
    df.drop('emp_title', axis=1, inplace=True)

# 4. emp_length (6.4% missing) - Fill with mode (most common value)
if df['emp_length'].isnull().any():
    df['emp_length'].fillna(df['emp_length'].mode()[0], inplace=True)

# 5. num_tl_120dpd_2m (5.2% missing) - Fill with 0 (no delinquencies)
df['num_tl_120dpd_2m'].fillna(0, inplace=True)

# 6. mo_sin_old_il_acct (2.9% missing) - Fill with median
df['mo_sin_old_il_acct'].fillna(df['mo_sin_old_il_acct'].median(), inplace=True)

# 7. percent_bc_gt_75 (1.1% missing) - Fill with median
df['percent_bc_gt_75'].fillna(df['percent_bc_gt_75'].median(), inplace=True)

# 8. bc_util (1.1% missing) - Fill with median
df['bc_util'].fillna(df['bc_util'].median(), inplace=True)

# 9. bc_open_to_buy (1.0% missing) - Fill with median
df['bc_open_to_buy'].fillna(df['bc_open_to_buy'].median(), inplace=True)

# 10. mths_since_recent_bc (1.0% missing) - Fill with median
df['mths_since_recent_bc'].fillna(df['mths_since_recent_bc'].median(), inplace=True)

# 11-14. Low missing (<0.1%) - Fill with median/mode
df['last_pymnt_d'].fillna(df['last_pymnt_d'].mode()[0], inplace=True)
df['revol_util'].fillna(df['revol_util'].median(), inplace=True)
df['last_credit_pull_d'].fillna(df['last_credit_pull_d'].mode()[0], inplace=True)
df['num_rev_accts'].fillna(df['num_rev_accts'].median(), inplace=True)

print("Missing values handled")
print(f"Remaining missing values: {df.isnull().sum().sum()}")
print(f"Shape: {df.shape}")

Missing values handled
Remaining missing values: 0
Shape: (87889, 91)


In [4]:
print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

# 1. Loan-to-Income ratio
df['loan_to_income'] = df['loan_amnt'] / df['annual_inc']

# 2. Installment-to-Income ratio (monthly payment burden)
df['installment_to_income'] = df['installment'] / (df['annual_inc'] / 12)

# 3. Credit age (convert earliest_cr_line to years)
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y', errors='coerce')
reference_date = pd.to_datetime('2018-12-31')
df['credit_history_years'] = ((reference_date - df['earliest_cr_line']).dt.days / 365.25)
df.drop('earliest_cr_line', axis=1, inplace=True)

print(f"\nFeature engineering complete")
print(f"Shape: {df.shape}")
print(f"\nNew features created:")
print("  1. loan_to_income")
print("  2. installment_to_income")
print("  3. credit_history_years")

FEATURE ENGINEERING

Feature engineering complete
Shape: (87889, 93)

New features created:
  1. loan_to_income
  2. installment_to_income
  3. credit_history_years


In [5]:
# Identify categorical columns that need encoding
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove target and ID columns from encoding
categorical_cols = [col for col in categorical_cols if col not in ['default']]

print(f"Categorical columns to encode: {len(categorical_cols)}")
print("\nCategorical columns:")
for col in categorical_cols:
    n_unique = df[col].nunique()
    print(f"  {col}: {n_unique} unique values")

Categorical columns to encode: 18

Categorical columns:
  term: 2 unique values
  grade: 7 unique values
  sub_grade: 35 unique values
  emp_length: 11 unique values
  home_ownership: 4 unique values
  verification_status: 3 unique values
  issue_d: 3 unique values
  pymnt_plan: 1 unique values
  purpose: 12 unique values
  zip_code: 878 unique values
  addr_state: 49 unique values
  initial_list_status: 2 unique values
  last_pymnt_d: 42 unique values
  last_credit_pull_d: 42 unique values
  application_type: 2 unique values
  hardship_flag: 1 unique values
  disbursement_method: 1 unique values
  debt_settlement_flag: 2 unique values


---
## Categorical Variables - Encoding Strategy

### Columns to Drop:
**Single-value columns (no information):**
- `pymnt_plan`: All same value
- `hardship_flag`: All same value  
- `disbursement_method`: All same value

**Date columns (not predictive for future loans):**
- `issue_d`: Loan issue date
- `last_pymnt_d`: Last payment date
- `last_credit_pull_d`: Last credit pull date

**Too granular:**
- `zip_code`: 878 unique values, `addr_state` captures geography better

### Columns to Encode:
**Ordinal (order matters):**
- `grade`: A → G (credit quality)
- `sub_grade`: A1 → G5 (finer credit quality)
- `emp_length`: Years of employment

**Nominal (no order):**
- `term`: 36 months / 60 months
- `home_ownership`: RENT, OWN, MORTGAGE, OTHER
- `verification_status`: Verified, Source Verified, Not Verified
- `purpose`: Loan purpose (debt consolidation, credit card, etc.)
- `addr_state`: US state
- `initial_list_status`: w (whole) / f (fractional)
- `application_type`: Individual / Joint
- `debt_settlement_flag`: Y / N

---

In [6]:
# Drop columns with single value (no variance)
single_value_cols = [col for col in categorical_cols if df[col].nunique() == 1]
print(f"Dropping {len(single_value_cols)} single-value columns:")
print(single_value_cols)
df.drop(columns=single_value_cols, inplace=True)

# Drop date columns (not predictive of future loans)
date_cols = ['issue_d', 'last_pymnt_d', 'last_credit_pull_d']
print(f"\nDropping {len(date_cols)} date columns:")
print(date_cols)
df.drop(columns=date_cols, inplace=True)

# Drop zip_code (too many unique values, addr_state captures geography)
print(f"\nDropping zip_code (too granular)")
df.drop(columns=['zip_code'], inplace=True)

# Update categorical list
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\n{'='*60}")
print(f"Remaining categorical columns to encode: {len(categorical_cols)}")
for col in categorical_cols:
    print(f"  {col}: {df[col].nunique()} unique values")
    
print(f"\nCurrent shape: {df.shape}")

Dropping 3 single-value columns:
['pymnt_plan', 'hardship_flag', 'disbursement_method']

Dropping 3 date columns:
['issue_d', 'last_pymnt_d', 'last_credit_pull_d']

Dropping zip_code (too granular)

Remaining categorical columns to encode: 11
  term: 2 unique values
  grade: 7 unique values
  sub_grade: 35 unique values
  emp_length: 11 unique values
  home_ownership: 4 unique values
  verification_status: 3 unique values
  purpose: 12 unique values
  addr_state: 49 unique values
  initial_list_status: 2 unique values
  application_type: 2 unique values
  debt_settlement_flag: 2 unique values

Current shape: (87889, 86)


In [7]:
from sklearn.preprocessing import LabelEncoder

# Ordinal encoding for ordered categories
ordinal_mappings = {
    'grade': {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7},
    'emp_length': {'< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, 
                   '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7,
                   '8 years': 8, '9 years': 9, '10+ years': 10}
}

for col, mapping in ordinal_mappings.items():
    df[col] = df[col].map(mapping)

# For sub_grade, extract numeric part (A1=1, A2=2, ... G5=35)
grade_map = {'A': 0, 'B': 5, 'C': 10, 'D': 15, 'E': 20, 'F': 25, 'G': 30}
df['sub_grade'] = df['sub_grade'].apply(lambda x: grade_map[x[0]] + int(x[1]))

print("Ordinal encoding complete")

# Label encoding for binary categories
binary_cols = ['term', 'initial_list_status', 'application_type', 'debt_settlement_flag']

for col in binary_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

print("Binary encoding complete")

# One-hot encoding for nominal categories with few values
nominal_cols = ['home_ownership', 'verification_status', 'purpose']

df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)

print("One-hot encoding complete")

# For addr_state (49 values), use frequency encoding
state_freq = df['addr_state'].value_counts() / len(df)
df['addr_state'] = df['addr_state'].map(state_freq)

print("State frequency encoding complete")

print(f"\nFinal shape: {df.shape}")
print(f"All columns are now numeric")

Ordinal encoding complete
Binary encoding complete
One-hot encoding complete
State frequency encoding complete

Final shape: (87889, 99)
All columns are now numeric


In [8]:
# Final data quality checks
print("FINAL DATA QUALITY CHECK")
print("="*60)

# Check for missing values
print(f"Missing values: {df.isnull().sum().sum()}")

# Check for infinite values
print(f"Infinite values: {np.isinf(df.select_dtypes(include=[np.number])).sum().sum()}")

# Check data types
print(f"\nData types:")
print(df.dtypes.value_counts())

# Verify target variable
print(f"\nTarget variable (default):")
print(f"  Class 0 (non-default): {(df['default'] == 0).sum():,} ({(df['default'] == 0).mean()*100:.2f}%)")
print(f"  Class 1 (default): {(df['default'] == 1).sum():,} ({(df['default'] == 1).mean()*100:.2f}%)")

print(f"\nFinal dataset shape: {df.shape}")

FINAL DATA QUALITY CHECK
Missing values: 0
Infinite values: 0

Data types:
float64    74
bool       16
int32       5
int64       4
Name: count, dtype: int64

Target variable (default):
  Class 0 (non-default): 70,287 (79.97%)
  Class 1 (default): 17,602 (20.03%)

Final dataset shape: (87889, 99)


In [9]:
# Remove data leakage features (known only AFTER loan outcome)
leakage_features = [
    'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 
    'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
    'last_pymnt_amnt', 'out_prncp', 'out_prncp_inv', 'last_fico_range_high', 'last_fico_range_low'
]

# Check which exist and remove them
leakage_present = [col for col in leakage_features if col in df.columns]

print(f"Removing {len(leakage_present)} data leakage features:")
for col in leakage_present:
    print(f"  - {col}")

df.drop(columns=leakage_present, inplace=True)

print(f"\nShape after removing leakage: {df.shape}")

Removing 12 data leakage features:
  - total_pymnt
  - total_pymnt_inv
  - total_rec_prncp
  - total_rec_int
  - total_rec_late_fee
  - recoveries
  - collection_recovery_fee
  - last_pymnt_amnt
  - out_prncp
  - out_prncp_inv
  - last_fico_range_high
  - last_fico_range_low

Shape after removing leakage: (87889, 87)


In [10]:
# Separate features and target
X = df.drop('default', axis=1)
y = df['default']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# First split: 70% train, 30% temp (for validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# Second split: Split temp into 50% validation, 50% test (15% each of total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("\n" + "="*60)
print("TRAIN/VALIDATION/TEST SPLIT")
print("="*60)
print(f"Training set:   {X_train.shape[0]:,} rows ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]:,} rows ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]:,} rows ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\n" + "="*60)
print("DEFAULT RATE IN EACH SET")
print("="*60)
print(f"Training:   {y_train.mean()*100:.2f}%")
print(f"Validation: {y_val.mean()*100:.2f}%")
print(f"Test:       {y_test.mean()*100:.2f}%")

Features shape: (87889, 86)
Target shape: (87889,)

TRAIN/VALIDATION/TEST SPLIT
Training set:   61,522 rows (70.0%)
Validation set: 13,183 rows (15.0%)
Test set:       13,184 rows (15.0%)

DEFAULT RATE IN EACH SET
Training:   20.03%
Validation: 20.03%
Test:       20.03%


In [11]:
# Scale numerical features (important for some models like Neural Networks, Logistic Regression)
scaler = StandardScaler()

# Fit on training data only
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames to keep column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Feature scaling complete")
print(f"Scaled training data shape: {X_train_scaled.shape}")
print(f"\nExample - Before scaling (first feature):")
print(f"  Mean: {X_train.iloc[:, 0].mean():.2f}")
print(f"  Std: {X_train.iloc[:, 0].std():.2f}")
print(f"\nAfter scaling (first feature):")
print(f"  Mean: {X_train_scaled.iloc[:, 0].mean():.6f}")
print(f"  Std: {X_train_scaled.iloc[:, 0].std():.6f}")

Feature scaling complete
Scaled training data shape: (61522, 86)

Example - Before scaling (first feature):
  Mean: 14367.06
  Std: 8597.46

After scaling (first feature):
  Mean: 0.000000
  Std: 1.000008


In [12]:
import pickle

# Save scaled data for modeling
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_val_scaled.to_csv('../data/processed/X_val_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)

y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

# Save unscaled data (for tree-based models that don't need scaling)
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_val.to_csv('../data/processed/X_val.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)

# Save scaler for future use
with open('../data/processed/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("All processed data saved")
print("\nSaved files:")
print("  - X_train.csv, X_val.csv, X_test.csv (unscaled)")
print("  - X_train_scaled.csv, X_val_scaled.csv, X_test_scaled.csv (scaled)")
print("  - y_train.csv, y_val.csv, y_test.csv (targets)")
print("  - scaler.pkl (for future scaling)")

All processed data saved

Saved files:
  - X_train.csv, X_val.csv, X_test.csv (unscaled)
  - X_train_scaled.csv, X_val_scaled.csv, X_test_scaled.csv (scaled)
  - y_train.csv, y_val.csv, y_test.csv (targets)
  - scaler.pkl (for future scaling)


---
## Preprocessing Complete

### Summary of Actions:

**1. Missing Value Handling:**
- Filled missing values using median/mode/domain logic
- Created flag for delinquency (48% missing = no delinquency)
- Dropped `emp_title` (too many unique values)

**2. Feature Engineering:**
- Created 3 new features:
  - `loan_to_income`: Loan burden relative to income
  - `installment_to_income`: Monthly payment affordability
  - `credit_history_years`: Length of credit history

**3. Data Cleaning:**
- Dropped single-value columns (no variance)
- Dropped date columns (not predictive)
- Dropped `zip_code` (too granular)

**4. Encoding:**
- Ordinal: `grade`, `sub_grade`, `emp_length`
- Binary: `term`, `initial_list_status`, `application_type`, `debt_settlement_flag`
- One-hot: `home_ownership`, `verification_status`, `purpose`
- Frequency: `addr_state`

**5. Data Split:**
- Training: 61,522 (70%)
- Validation: 13,183 (15%)
- Test: 13,184 (15%)
- Stratified: 20.03% default rate in all sets

**6. Feature Scaling:**
- StandardScaler (mean=0, std=1)
- Saved both scaled and unscaled versions

**7. Data Leakage Prevention:**
- Removed 10 features known only after loan outcome:
  - Payment amounts (total_pymnt, total_rec_prncp, etc.)
  - Recovery amounts (recoveries, collection_recovery_fee)
  - Outstanding principal (out_prncp, out_prncp_inv)

### Ready for Modeling:
- 88 features, all numeric (removed 10 data leakage features)
- No missing values
- No infinite values
- Balanced class distribution preserved
- **Data leakage removed:** Dropped post-outcome features (total_pymnt, recoveries, etc.)

---